Leveraging LLM Reasoning Enhances Personalized Recommender Systems(2024)에서 제안된 LLM을 이용한 추천시스템 제작 방안임

[필요한 파일]

총 6개

대출상품별 데이터 : [**products_name**]_loan_data.csv

대출상품별 사용자의 데이터 : [**products_name**]_user_data.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 사용자별 데이터 가져오기

각 대출 상품별로 필요한 컬럼들의 조합을 다르게 한 사용자들의 데이터

In [ ]:
import pandas as pd

credit_user_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/credit_user_data.csv')
mortgate_user_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/mortgate_user_data.csv')
rent_user_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/rent_user_data.csv')

print(f'credit_user_df shape : {credit_user_df.shape}')
print(f'mortgate_user_df shape : {mortgate_user_df.shape}')
print(f'rent_user_df shape : {rent_user_df.shape}')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BOAZ/mini_project_2/credit_user_data.csv'

## 2. 대출상품데이터 데이터 가져오기

In [ ]:
credit_loan_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/credit_loan_data.csv')
mortgate_loan_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/mortgate_loan_data.csv')
rent_loan_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/rent_loan_data.csv')

print(f'credit_loan_df shape : {credit_loan_df.shape}')
print(f'mortgate_loan_df shape : {mortgate_loan_df.shape}')
print(f'rent_loan_df shape : {rent_loan_df.shape}')

NameError: name 'ast' is not defined

## 3. UserData와 LoanProductData를 LoanApproved value에 기반하여 reasoning하기

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai

GOOGLE_API_KEY = ''
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
def generate_reasoning(loan_type, user_data, loan_data):
    base_prompt = f"Base on given user's metadata {user_data}, Will user be able to get approved for following loan product? \n\n [Loan Product's Metadata]\n{loan_data}\n\n"
    description_prompt = "We will also provide you the ground truth that user has been approved for a loan.\nWe want you to generate a reasonable explanation with given informations\n"
    optional_prompt = None

    if loan_type == 'credit':
        optional_prompt = "In particular, create an explanation focusing on the values ​​of the 'CreditScore' and 'LoanAmount'."
    else:
        optional_prompt = "In particular, create an explanation focusing on the values ​​of the 'HomeOwnerShipStatus' and 'LoanAmount'."

    reasoning_prompt = f"{base_prompt}{description_prompt}{optional_prompt}"

    reasoning_response = model.generate_text(prompt=reasoning_prompt)

    return reasoning_response

In [ ]:
RecSAVER_credit = {
    'reasoning_sentences': []
    'loan_approved' : []
}

In [ ]:
RecSAVER_mortgage = {
    'reasoning_sentences': []
    'loan_approved' : []
}

In [ ]:
RecSAVER_rent = {
    'reasoning_sentences': []
    'loan_approved' : []
}

In [ ]:
## 개인신용대출 reasoning
for _, user_row in credit_user_df.iterrows():
    for _, loan_row in credit_loan_df.iterrows():
        user_data = user_row.to_dict()
        loan_data = loan_row.to_dict()

        reasoning_response = generate_reasoning('credit', user_data, loan_data)
        loan_approved = user_row['LoanApproved']

        RecSAVER_credit['reasoning_sentences'].append(reasoning_response)
        RecSAVER_credit['loan_approved'].append(loan_approved)

In [ ]:
## 주택담보대출 reasoning
for _, user_row in mortgage_user_df.iterrows():
    for _, loan_row in mortgate_loan_df.iterrows():
        user_data = user_row.to_dict()
        loan_data = loan_row.to_dict()

        reasoning_response = generate_reasoning('mortgage', user_data, loan_data)
        loan_approved = user_row['LoanApproved']

        RecSAVER_mortgage['reasoning_sentences'].append(reasoning_response)
        RecSAVER_mortgage['loan_approved'].append(loan_approved)

In [ ]:
## 전세자금대출 reasoning
for _, user_row in rent_user_df.iterrows():
    for _, loan_row in rent_loan_df.iterrows():
        user_data = user_row.to_dict()
        loan_data = loan_row.to_dict()

        reasoning_response = generate_reasoning('rent', user_data, loan_data)
        loan_approved = user_row['LoanApproved']

        RecSAVER_rent['reasoning_sentences'].append(reasoning_response)
        RecSAVER_rent['loan_approved'].append(loan_approved)